In [1]:
import sklearn

#NN Surrogate model class
from injector_surrogate_quads import *

from emittance_calc import get_normemit
from sampling_functions import get_ground_truth, get_beamsize

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#BO
from bayes_opt import BayesianOptimization

# Load injector model

In [3]:
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

energy = 0.135 #GeV

# Set up sampling and objectives

In [4]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

In [5]:
ref_point = ref_point[0]

In [30]:
#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','SQ01:b1_gradient','CQ01:b1_gradient','QE04:b1_gradient']

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(varx,vary,varz,varscan=[-4.4, -3.5, -3, -2.5, -2.3]): 
    numscan = len(varscan)
    
    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((numscan,len(Model.model_in_list)))
    
    #fill in reference point around which to optimize
    for i in range(0,len(ref_point)):
        x_in[:,i]=ref_point[i] 

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz
    
    #set quad 525 to values for scan
    x_in[:, Model.loc_in[opt_var_names[3]]] = varscan

    #output predictions
    y_out = Model.pred_machine_units(x_in) 
    
    emitx = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    emity = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    
    print(f"{np.mean(emitx)/1e-6:.2f}, {np.mean(emity)/1e-6:.2f}")
    #plt.plot(x_in[:,Model.loc_in['QE04:b1_gradient']],y_out[:,0], label="sig_x")
    #plt.plot(x_in[:,Model.loc_in['QE04:b1_gradient']],y_out[:,1], label="sig_y")
    
    x_rms = y_out[:,0]
    y_rms = y_out[:,1]

    norm_emit = get_normemit(energy, varscan, x_rms, y_rms) #normalized emittance

    # return emit predicted, norm geo emit calc, norm emit x and y
    #return -1*objective(y_out)[0], -np.sqrt(emitx*emity)/1e-6, emitx/1e-6, emity/1e-6
#     if np.isnan(np.sqrt(emitx*emity)):
#         return -1000
    return  -norm_emit/1e-6


# def objective(y_out):
    
#     #output is geometric emittance in transverse plane
#     out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
#     out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    
#     return np.sqrt(out1*out2)/1e-6 # in um units

In [24]:
# x axis works around focus
evaluate(0.4657, -0.0071, -0.0169, [-3.9, -3.5, -3, -2.5, -2.3])



1.2638099633477395 1.5351677120634122


-5.697039549150763

In [10]:
evaluate(0.4657, -0.0071, -0.0169, [  -6.5, -6,  -5, -4.5, -4, -3, -2.5])


1.281426534660568 1.5350985904660774


-11.814177865098927

In [11]:
evaluate( 0.4657  , -0.007063 , -0.01989, [-7,-6,  -5, -4, -3, -2, 0 ])

1.3624251096189255 1.6438663124063169


-1000000000.0


# Simple BO 

In [31]:
# bounds on input params 
pbounds = {'varx': (0.44, 0.55),
           'vary': (-0.02, 0.02),
           'varz': (-0.02, 0.02),
          }

optimizer = BayesianOptimization(
    f = evaluate,
    pbounds = pbounds,
    random_state = 1,
)

optimizer.maximize(
    init_points=5,
    n_iter=40,
)

|   iter    |  target   |   varx    |   vary    |   varz    |
-------------------------------------------------------------
0.94, 1.41
|  1        | -4.905    |  0.4859   |  0.008813 | -0.02     |
1.04, 1.17
|  2        | -2.283    |  0.4733   | -0.01413  | -0.01631  |
1.14, 1.28
|  3        | -13.41    |  0.4605   | -0.006178 | -0.004129 |
2.43, 1.66
|  4        | -17.13    |  0.4993   | -0.003232 |  0.007409 |
0.88, 1.22
|  5        | -10.55    |  0.4625   |  0.01512  | -0.0189   |
0.89, 0.99
|  6        | -1e+09    |  0.4813   |  0.0116   |  0.01098  |
0.78, 0.98
|  7        | -1.711    |  0.4817   |  0.01012  | -0.01978  |
3.45, 2.43
|  8        | -11.73    |  0.5049   | -0.0159   | -0.008553 |
1.17, 0.83
|  9        | -5.516    |  0.465    | -0.01378  |  0.01789  |
4.81, 1.72
|  10       | -49.84    |  0.5157   | -0.01448  |  0.01287  |
1.35, 1.23
|  11       | -1e+09    |  0.4917   | -0.00306  | -0.001123 |
3.61, 1.81
|  12       | -36.09    |  0.5072   | -0.006512 |  0.0121   |


## Results from simple BO

In [6]:
SOL_opt = optimizer.max['params']['varx'] # solenoid val at optimum
CQ_opt = optimizer.max['params']['vary'] # CQ val at optimum
SQ_opt = optimizer.max['params']['varz'] # SQ val at optimum

opt_emit = -1*optimizer.max['target'] # emittance value at optimum (in um)

print('optimum (pv_units) ',SOL_opt, CQ_opt, SQ_opt)
print('optimum geom emit ', opt_emit)

optimum (pv_units)  0.4783371750558134 -0.02 0.02
optimum geom emit  0.8672671914100647
